In [ ]:
PYOLITE = False
try:
    import micropip
    await micropip.install(["pyoliteutils","scapy"])
    PYOLITE = True
except:
    pass 

In [ ]:
if PYOLITE:
    from pyoliteutils import *
    GITFOLDERURL = "https://raw.githubusercontent.com/UTCSheffield/Y10-Computing-GCSE-jupyter/main/content/Firewall/"
    await get_file_from_url(GITFOLDERURL+"whitelist.txt")
    await get_file_from_url(GITFOLDERURL+"blacklist.txt")
    await get_file_from_url(GITFOLDERURL+"logs.txt")

In [ ]:
import os
import sys
import time
from collections import defaultdict
from scapy.all import sniff, IP, TCP

In [ ]:
THRESHOLD = 40
DEBUG = True
LOGFILE = "logs.txt"
DEFAULTALLOW = True
LINUX = False

print(f"THRESHOLD: {THRESHOLD}")

# Read IPs from a file
def read_ip_file(filename):
    with open(filename, "r") as file:
        ips = [line.strip() for line in file]
    return set(ips)

# Check for Nimda worm signature
def is_nimda_worm(packet):
    if packet.haslayer(TCP) and packet[TCP].dport == 80:
        payload = packet[TCP].payload
        return "GET /scripts/root.exe" in str(payload)
    return False

# Log events to a file
def log_event(message):
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
    
    
    with open(LOGFILE, "a") as file:
        file.write(f"{message}\n")
    if (DEBUG):
        print(message)

'''
# Log events to a file
def log_event(message):
    log_folder = "logs"
    os.makedirs(log_folder, exist_ok=True)
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
    log_file = os.path.join(log_folder, f"log_{timestamp}.txt")
    
    with open(log_file, "a") as file:
        file.write(f"{message}\n")
'''

def packet_callback(packet):
    src_ip = packet[IP].src

      # Check if IP is in the whitelist
    if src_ip in whitelist_ips:
        return True

    # Check if IP is in the blacklist
    if src_ip in blacklist_ips:
        if LINUX:
            os.system(f"iptables -A INPUT -s {src_ip} -j DROP")
        log_event(f"Blocking blacklisted IP: {src_ip}")
        return False
    
      # Check for Nimda worm signature
    if is_nimda_worm(packet):
        print(f"Blocking Nimda source IP: {src_ip}")
        if LINUX:
            os.system(f"iptables -A INPUT -s {src_ip} -j DROP")
        log_event(f"Blocking Nimda source IP: {src_ip}")
        return False

    packet_count[src_ip] += 1

    current_time = time.time()
    time_interval = current_time - start_time[0]

    if time_interval >= 1:
        for ip, count in packet_count.items():
            packet_rate = count / time_interval

            if packet_rate > THRESHOLD and ip not in blocked_ips:
                print(f"Blocking IP: {ip}, packet rate: {packet_rate}")
                if LINUX:
                    os.system(f"iptables -A INPUT -s {ip} -j DROP")
                log_event(f"Blocking IP: {ip}, packet rate: {packet_rate}")
                blocked_ips.add(ip)
                return False

        packet_count.clear()
        start_time[0] = current_time
    if DEFAULTALLOW :
        log_event(f"Allowing IP: {src_ip}")
        return True
    log_event(f"Blocking IP: {src_ip}")
    return False #True

if __name__ == "__main__":
    if LINUX and os.geteuid() != 0:
        print("This script requires root privileges.")
        sys.exit(1)

    # Import whitelist and blacklist IPs
    whitelist_ips = read_ip_file("whitelist.txt")
    blacklist_ips = read_ip_file("blacklist.txt")

    packet_count = defaultdict(int)
    start_time = [time.time()]
    blocked_ips = set()

    print("Monitoring network traffic...")
    #sniff(filter="ip", prn=packet_callback)

    

In [ ]:
start_time = [time.time()]

## Whitelist
assert packet_callback(IP(src="127.0.0.1", dst="127.0.0.1")/TCP(dport=666)) == True

## Blacklist
assert packet_callback(IP(src="198.51.100.255", dst="127.0.0.1")/TCP(dport=666)) == False


## Not on a list
assert packet_callback(IP(src="127.0.0.2", dst="127.0.0.1")/TCP(dport=666)) == DEFAULTALLOW

## Not on a list but about to DoS us
assert packet_callback(IP(src="127.0.0.3", dst="127.0.0.1")/TCP(dport=666)) == DEFAULTALLOW

# DoSing
for i in range(50):
    packet_callback(IP(src="127.0.0.3", dst="127.0.0.1")/TCP(dport=666))

time.sleep(1)
# Has DoSed us
assert packet_callback(IP(src="127.0.0.3", dst="127.0.0.1")/TCP(dport=666)) == False


In [ ]:
with open(LOGFILE) as f:
    print(f.read())

In [ ]:
assert packet_callback(IP(src="127.0.0.3", dst="127.0.0.1")/TCP(dport=666)) == False